In [101]:
import pynini
from pynini.lib import paradigms, features, rewrite, pynutil

In [102]:
tira_consonants = [
    'm', 'n',      'ɲ', 'ŋ',
    'p', 't̪', 't', 'c', 'k',
    'b', 'd̪', 'd', 'ɟ', 'g',
              's',
    'v', 'ð',
    'w',      'l'  'j',
         'r', 'ɾ', 'ɽ',
]

tira_vowels = [
    'i',      'u',
    'ɪ',      'ʊ',
    'e', 'ə', 'o',
    'ɛ', 'ɜ', 'ɔ',
         'a',
]

high_tone = '\u0301'
low_tone = '\u0300'
fall_tone = '\u0306'
rise_tone = '\u030c'
tira_tones = [high_tone, low_tone, fall_tone, rise_tone]

tira_symbol_table = pynini.SymbolTable(name="Tira phones")
for phone in tira_consonants+tira_vowels+tira_tones:
    tira_symbol_table.add_symbol(phone)
tira_symbol_table

<SymbolTable 'Tira phones' at 0x103cc3e70>

In [103]:
C = pynini.union(*tira_consonants).optimize()
V = pynini.union(*tira_vowels).optimize()
T = pynini.union(*tira_tones).optimize()
SigmaStar = pynini.union(C,V,T).closure().optimize()

In [104]:
def print_forms(stem: str, paradigm: paradigms.Paradigm):
    lattice = rewrite.rewrite_lattice(
        stem,
        paradigm.stems_to_forms @ paradigm.feature_label_rewriter
    )
    for wordform in rewrite.lattice_to_strings(lattice):
        print(wordform)

In [105]:
verb_roots = [
    "vɛð",
    "mɛð",
    "ap",
    "ad",
]
stem = paradigms.make_byte_star_except_boundary()
class_prefixes = [
    "j",
    "g",
    "t̪",
    "ð",
    "n",
    "ɲ",
    "ŋ",
    "r",
    "l",
]

In [106]:
tam = features.Feature(
    "tam",
    "imperfective",
    "perfective",
    "dependent",
    "infinitive",
    "imperative"
)
deixis = features.Feature("deixis", "ventive", "itive")
class_agree = features.Feature("class", *class_prefixes)

inflected_verb = features.Category(tam, deixis, class_agree)

In [107]:
VO_VENT_FV = lambda stem: paradigms.suffix("+ɔ́", stem)

OO_IT_IPFV_FV = lambda stem: paradigms.suffix("+ɔ̀", stem)
AO_IT_IPFV_FV = lambda stem: paradigms.suffix("+à", stem)
VO_IT_PFV_FV = lambda stem: paradigms.suffix("+ɛ̀", stem)

In [108]:
itive_d_stem = C.closure() + V + pynutil.insert(high_tone) + stem.closure()
ventive_d_stem = C.closure() + V + pynutil.insert(low_tone) + stem.closure()
itive_d_stem = itive_d_stem.optimize()
ventive_d_stem = ventive_d_stem.optimize()

ipfv_aux = paradigms.prefix("á+", stem.closure())
pfv_it_aux = paradigms.prefix("à+", stem.closure())
pfv_vent_schwa = paradigms.prefix("ə̀+", stem.closure())

VO_IT_PFV_FV(verb_roots[0]@itive_d_stem).string()

'vɛ́ð+ɛ̀'

In [109]:
HarmonyTarget = pynini.union("ɛ", "a", "ɜ").optimize()
HarmonyChange = pynini.cross(HarmonyTarget, "ɔ")
HarmonyTrigger = "ɔ"
rounding_harmony = pynini.cdrewrite(HarmonyChange, SigmaStar, SigmaStar+HarmonyTrigger, SigmaStar)
("vɛ̀ðɔ́" @ rounding_harmony).string()

'vɔ̀ðɔ́'

In [110]:
ipfv_it = features.FeatureVector(inflected_verb, "tam=imperfective", "deixis=itive")
ipfv_vent = features.FeatureVector(inflected_verb, "tam=imperfective", "deixis=ventive")
pfv_it = features.FeatureVector(inflected_verb, "tam=perfective", "deixis=itive")
pfv_vent = features.FeatureVector(inflected_verb, "tam=perfective", "deixis=ventive")

AO_IPFV_SLOTS = [
    (AO_IT_IPFV_FV(itive_d_stem@ipfv_aux), ipfv_it),
    (VO_VENT_FV(ventive_d_stem@ipfv_aux), ipfv_vent),
]
OO_IPFV_SLOTS = [
    (OO_IT_IPFV_FV(itive_d_stem@ipfv_aux), ipfv_it),
    (VO_VENT_FV(ventive_d_stem@ipfv_aux), ipfv_vent),
]
VO_PFV_SLOTS = [
    (VO_IT_PFV_FV(itive_d_stem@pfv_it_aux), pfv_it),
    (VO_VENT_FV(ventive_d_stem@pfv_vent_schwa), pfv_vent)
]

def add_class_prefixes_to_slots(slot_list):
    slots_w_class_prefixes = []
    for stem, feature_vector in slot_list:
        category = feature_vector.category
        feature_values = [f"{feature}={value}" for feature, value in feature_vector.values.items()]
        for prefix in class_prefixes:
            features_with_class = features.FeatureVector(category, f"class={prefix}", *feature_values)
            prefixed_verb = paradigms.prefix(f"{prefix}+", stem)
            slots_w_class_prefixes.append((prefixed_verb, features_with_class))
    return slots_w_class_prefixes

AO_IPFV_SLOTS = add_class_prefixes_to_slots(AO_IPFV_SLOTS)
OO_IPFV_SLOTS = add_class_prefixes_to_slots(OO_IPFV_SLOTS)
VO_PFV_SLOTS = add_class_prefixes_to_slots(VO_PFV_SLOTS)

In [111]:
lemma_verb_features = features.FeatureVector(inflected_verb, "tam=imperfective", "deixis=itive", "class=g")
AO = paradigms.Paradigm(
    category=inflected_verb,
    name="AO FV class indicative forms",
    slots=AO_IPFV_SLOTS+VO_PFV_SLOTS,
    lemma_feature_vector=lemma_verb_features,
    stems=verb_roots,
)
print_forms(verb_roots[0], AO)

ɲ+ə̀+vɛ̀ð+ɔ́[class=ɲ][deixis=ventive][tam=perfective]
ɲ+á+vɛ́ð+à[class=ɲ][deixis=itive][tam=imperfective]
ɲ+á+vɛ̀ð+ɔ́[class=ɲ][deixis=ventive][tam=imperfective]
ɲ+à+vɛ́ð+ɛ̀[class=ɲ][deixis=itive][tam=perfective]
ŋ+ə̀+vɛ̀ð+ɔ́[class=ŋ][deixis=ventive][tam=perfective]
ŋ+á+vɛ́ð+à[class=ŋ][deixis=itive][tam=imperfective]
ŋ+á+vɛ̀ð+ɔ́[class=ŋ][deixis=ventive][tam=imperfective]
ŋ+à+vɛ́ð+ɛ̀[class=ŋ][deixis=itive][tam=perfective]
ð+ə̀+vɛ̀ð+ɔ́[class=ð][deixis=ventive][tam=perfective]
ð+á+vɛ́ð+à[class=ð][deixis=itive][tam=imperfective]
ð+á+vɛ̀ð+ɔ́[class=ð][deixis=ventive][tam=imperfective]
ð+à+vɛ́ð+ɛ̀[class=ð][deixis=itive][tam=perfective]
t̪+ə̀+vɛ̀ð+ɔ́[class=t̪][deixis=ventive][tam=perfective]
t̪+á+vɛ́ð+à[class=t̪][deixis=itive][tam=imperfective]
t̪+á+vɛ̀ð+ɔ́[class=t̪][deixis=ventive][tam=imperfective]
t̪+à+vɛ́ð+ɛ̀[class=t̪][deixis=itive][tam=perfective]
r+ə̀+vɛ̀ð+ɔ́[class=r][deixis=ventive][tam=perfective]
r+á+vɛ́ð+à[class=r][deixis=itive][tam=imperfective]
r+á+vɛ̀ð+ɔ́[class=r

In [130]:
# lattice = rewrite.rewrite_lattice(
#     "vɛð",
#     AO.stems_to_forms
# )
# for wordform in rewrite.lattice_to_strings(lattice):
#     print(wordform)
inflected_verb_str = "gə̀vɛ̀ðɔ́"
AO.lemmatize(inflected_verb_str)
# (inflected_verb_str @ AO.lemmatizer).string()

[('gávɛ́ðà',
  FeatureVector(Category(Feature('class', 'j', 'g', 't̪', 'ð', 'n', 'ɲ', 'ŋ', 'r', 'l'), Feature('deixis', 'ventive', 'itive'), Feature('tam', 'imperfective', 'perfective', 'dependent', 'infinitive', 'imperative')), 'class=g', 'deixis=ventive', 'tam=perfective'))]